In [36]:
import os
import time

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter


from PIL import Image

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tqdm import tqdm

### Metadata

In [37]:
meta = pd.read_csv("data/HAM10000_metadata.csv")
meta.head()

### Link Metadata to Images

In [38]:
def get_image_path(image_id):
    if os.path.exists(os.path.join(image_dir_1, image_id + ".jpg")):
        return os.path.join(image_dir_1, image_id + ".jpg")
    elif os.path.exists(os.path.join(image_dir_2, image_id + ".jpg")):
        return os.path.join(image_dir_2, image_id + ".jpg")
    else:
        return None

In [39]:
image_dir_1 = "data/HAM10000_images_part_1/"
image_dir_2 = "data/HAM10000_images_part_2/"

df = meta
df["image_path"] = df["image_id"].apply(get_image_path)

missing_images = df[df["image_path"].isna()]
print(f"Missing images: {len(missing_images)}")

df.head()

### Understand Class Distribution

- Actinic keratoses and intraepithelial carcinoma / Bowen's disease (**akiec**), 
- basal cell carcinoma (**bcc**), 
- benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, (**bkl**), 
- dermatofibroma (**df**), 
- melanoma (**mel**), 
- melanocytic nevi (**nv**) and 
- vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, (**vasc**).

In [40]:
class_counts = df["dx"].value_counts()

plt.figure(figsize=(10, 5))
sns.barplot(x=class_counts.index, y=class_counts.values, palette="viridis")
plt.title("Class Distribution in the HAM10000 Dataset")
plt.xlabel("Skin Lesion Type")
plt.ylabel("Number of Images")
plt.xticks(rotation=30)
plt.show()

print("Class Distribution:")
print(class_counts)

We'll have to factor in this during the modeling iterations and perhaps add a method to account for it.

### Data Preprocessing (Resizing & Normalising) and Splitting

In [41]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for efficiency
    transforms.RandomHorizontalFlip(p=0.5),  # Flip images randomly (lesions don't have strict orientation)
    transforms.RandomRotation(degrees=20),  # Slight rotation (lesions appear at different angles)
#     transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Vary lighting conditions
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small position shifts
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [42]:
class HAM10000Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]["image_path"]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        label = self.dataframe.iloc[idx]["dx"]

        label_map = {'akiec': 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv': 5, 'vasc': 6}
        label_idx = label_map[label]

        return image, label_idx

In [43]:
# sub = df.sample(2000)
sub = df

unique_lesions = sub["lesion_id"].unique()
train_lesions, val_lesions = train_test_split(unique_lesions, test_size=0.2, random_state=42)
train_df = sub[sub["lesion_id"].isin(train_lesions)]
val_df = sub[sub["lesion_id"].isin(val_lesions)]

print(f"Train Set: {len(train_df)} images")
print(f"Validation Set: {len(val_df)} images")

In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

In [45]:
batch_size = 64
train_dataset = HAM10000Dataset(train_df, transform=transform)
val_dataset = HAM10000Dataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


print(f"Using device: {device}")
print(f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")

### Model Selection and Training Setup

In [46]:
num_classes = 7
# model = models.resnet34(pretrained=True)
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)


model = model.to(device)
model

In [47]:
class_counts = df["dx"].value_counts().sort_index()
total_samples = len(df)

class_weights = total_samples / (len(class_counts) * class_counts)  # inverse frequency
class_weights = torch.tensor(class_weights.values, dtype=torch.float32).to(device)

# loss function with class weights
criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
print("Class Weights (Used in Loss Function):", class_weights)

In [48]:
optimiser = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-6)
# optimiser = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# lr scheduler - reduces LR when validation loss plateaus
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size=5, gamma=0.1)

### Model Training

In [49]:
def train_model(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

        progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    avg_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy



def validate_model(model, val_loader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    progress_bar = tqdm(val_loader, desc=f"Validating Epoch {epoch+1}", leave=False)

    with torch.no_grad():
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix(loss=loss.item(), acc=100 * correct / total)

    avg_loss = running_loss / len(val_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy



In [52]:
for images, labels in train_loader:
    print("Loaded a batch of images:", images.shape)
    break

Loaded a batch of images: torch.Size([64, 3, 224, 224])


In [53]:
patience = 3
no_improve_epochs = 0
num_epochs = 10
best_val_acc = 0.0
save_path = "best_renet34_10epochs_notpretrained.pth"

writer = SummaryWriter(log_dir="runs/HAM10000_resnet50")


for epoch in range(num_epochs):
    start_time = time.time()

    print(f"\nEpoch {epoch+1}/{num_epochs}")

    train_loss, train_acc = train_model(model, train_loader, criterion, optimiser, device, epoch)
    val_loss, val_acc = validate_model(model, val_loader, criterion, device, epoch)

    writer.add_scalar("Loss/Train", train_loss, epoch)
    writer.add_scalar("Loss/Validation", val_loss, epoch)
    writer.add_scalar("Accuracy/Train", train_acc, epoch)
    writer.add_scalar("Accuracy/Validation", val_acc, epoch)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1

    end_time = time.time()
    epoch_time = end_time - start_time

    print(f"\nEpoch [{epoch+1}/{num_epochs}] - Time: {epoch_time:.2f}s")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print("-" * 50)

    if no_improve_epochs >= patience:
        print(f"Early stopping triggered at epoch {epoch+1} 🚨")
        break

writer.close()
print("Training complete. Best validation accuracy:", best_val_acc)



Epoch 1/10



Epoch [1/10] - Time: 214.51s
Train Loss: 0.4091, Train Acc: 79.72%
Val Loss: 0.8199, Val Acc: 74.96%
--------------------------------------------------

Epoch 2/10



Epoch [2/10] - Time: 209.10s
Train Loss: 0.3945, Train Acc: 81.09%
Val Loss: 0.8409, Val Acc: 73.49%
--------------------------------------------------

Epoch 3/10



Epoch [3/10] - Time: 210.06s
Train Loss: 0.3406, Train Acc: 82.72%
Val Loss: 0.6929, Val Acc: 77.32%
--------------------------------------------------

Epoch 4/10



Epoch [4/10] - Time: 213.36s
Train Loss: 0.3364, Train Acc: 84.25%
Val Loss: 0.7150, Val Acc: 75.89%
--------------------------------------------------

Epoch 5/10


...


Epoch [6/10] - Time: 221.34s
Train Loss: 0.2503, Train Acc: 86.07%
Val Loss: 0.7677, Val Acc: 79.72%
--------------------------------------------------

Epoch 7/10



Epoch [7/10] - Time: 218.66s
Train Loss: 0.1921, Train Acc: 88.83%
Val Loss: 0.7344, Val Acc: 78.29%
--------------------------------------------------

Epoch 8/10



Epoch [8/10] - Time: 216.99s
Train Loss: 0.2105, Train Acc: 88.19%
Val Loss: 0.7366, Val Acc: 80.60%
--------------------------------------------------

Epoch 9/10



Epoch [9/10] - Time: 217.91s
Train Loss: 0.2630, Train Acc: 86.97%
Val Loss: 0.6332, Val Acc: 81.87%
--------------------------------------------------

Epoch 10/10



Epoch [10/10] - Time: 218.32s
Train Loss: 0.1866, Train Acc: 90.12%
Val Loss: 0.7230, Val Acc: 79.47%
--------------------------------------------------
Training complete. Best validation accuracy: 81.87163155316021


### Model Evaluation

In [54]:
def evaluate_model(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_labels, all_preds

def classification_report_df(true_labels, predicted_labels, class_names):
    report_dict = classification_report(true_labels, predicted_labels, target_names=class_names, output_dict=True)
    
    report_df = pd.DataFrame(report_dict).transpose()
    report_df = report_df.round(4)
    
    return report_df

In [55]:
true_labels, predicted_labels = evaluate_model(model, val_loader, device)

In [56]:
class_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
report_df = classification_report_df(true_labels, predicted_labels, class_names)
report_df

,precision,recall,f1-score,support
akiec,0.5857,0.6029,0.5942,68.0000
bcc,0.7748,0.6515,0.7078,132.0000
bkl,0.6226,0.8115,0.7046,244.0000
df,0.5250,0.6176,0.5676,34.0000
mel,0.5119,0.6711,0.5808,225.0000
nv,0.9424,0.8418,0.8892,1302.0000
vasc,0.7500,0.9167,0.8250,36.0000
accuracy,0.7967,0.7967,0.7967,0.7967
macro avg,0.6732,0.7304,0.6956,2041.0000
weighted avg,0.8236,0.7967,0.8051,2041.0000


In [57]:
report_df.to_csv("resnet50_10epochs_class_scores.csv")

### Inference - on the test set
- Not interested right now.

In [58]:
def predict_image(image_path, model, device):
    model.eval()
    
    image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)
    
    class_map = {0: 'akiec', 1: 'bcc', 2: 'bkl', 3: 'df', 4: 'mel', 5: 'nv', 6: 'vasc'}
    return class_map[predicted_class.item()]

In [59]:
image_path = "data/HAM10000_images_part_1/ISIC_0027419.jpg"
predicted_class = predict_image(image_path, model, device)
predicted_class

'bkl'

In [ ]:
# 1k; bs 64 - 1m 48.4s
# 2k; bs 64 - 3m 56s